<a href="https://colab.research.google.com/github/karanjavali/SER531-Team23-Fall2023/blob/main/preprocessing_notebooks/HateCrime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/531-Datasets/HateCrime/HateCrime2.csv')

# Display the first few rows of the DataFrame
df.head()

,REC_IR,STATNUM,ORI,ORIADDDT,ORINIBRS,CITY,STATECOD,POPGRP,DIVISN,REGION,...,VTYP_O9,VTYP_U9,VTYP_I10,VTYP_B10,VTYP_F10,VTYP_G10,VTYP_R10,VTYP_S10,VTYP_O10,VTYP_U10
0,IR,50,AK0010100,20191130,0,Anchorage,AK,1C,9,4,...,0,0,0,0,0,0,0,0,0,0
1,IR,50,AK0010300,20191130,0,Juneau,AK,4,9,4,...,0,0,0,0,0,0,0,0,0,0
2,IR,50,AK0010300,20191130,0,Juneau,AK,4,9,4,...,0,0,0,0,0,0,0,0,0,0
3,IR,50,AKAST0100,20191130,0,Anchorage,AK,8A,9,4,...,0,0,0,0,0,0,0,0,0,0
4,IR,1,AL0010200,20191130,0,Birmingham,AL,2,6,3,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Assuming df is your DataFrame
selected_columns = ['GOFFRAC', 'OFFCOD1', 'OFFCOD2', 'OFFCOD3', 'OFFCOD4', 'OFFCOD5',
                     'OFFCOD6', 'OFFCOD7', 'OFFCOD8', 'OFFCOD9', 'OFFCOD10', 'LOCCOD1', 'LOCCOD2', 'LOCCOD3',
                     'LOCCOD4', 'LOCCOD5', 'LOCCOD6', 'LOCCOD7', 'LOCCOD8', 'LOCCOD9', 'LOCCOD10', 'BIASMO1',
                     'BIASMO2', 'BIASMO3', 'BIASMO4', 'BIASMO5', 'BIASMO6', 'BIASMO7', 'BIASMO8', 'BIASMO9',
                     'BIASMO10', 'CITY', 'STATECOD']

# Add other columns as needed
# selected_columns.extend(['Other Column 1', 'Other Column 2', ...])

# Create a new DataFrame with only the selected columns
df_selected = df[selected_columns]
#df_selected.head()

In [ ]:
def concat_non_null(row, prefix):
    # Extract values from columns with the specified prefix
    values = [str(row[col]) for col in row.index if col.startswith(prefix)]

    # Exclude empty strings from the concatenation
    values = [value for value in values if value != '']
    #print(values)
    # Concatenate non-empty values with commas
    result = values[0]

    return result

# Create a copy of the DataFrame to avoid SettingWithCopyWarning
df_selected = df_selected.copy()

# Create new columns by concatenating the specified columns
df_selected['OFFCOD'] = df_selected.apply(lambda row: concat_non_null(row, 'OFFCOD'), axis=1)
df_selected['LOCCOD'] = df_selected.apply(lambda row: concat_non_null(row, 'LOCCOD'), axis=1)
df_selected['BIASMO'] = df_selected.apply(lambda row: concat_non_null(row, 'BIASMO'), axis=1)

# Drop the original columns
df_selected.drop(columns=['OFFCOD1', 'OFFCOD2', 'OFFCOD3', 'OFFCOD4', 'OFFCOD5', 'OFFCOD6', 'OFFCOD7', 'OFFCOD8', 'OFFCOD9', 'OFFCOD10',
                           'LOCCOD1', 'LOCCOD2', 'LOCCOD3', 'LOCCOD4', 'LOCCOD5', 'LOCCOD6', 'LOCCOD7', 'LOCCOD8', 'LOCCOD9', 'LOCCOD10',
                           'BIASMO1', 'BIASMO2', 'BIASMO3', 'BIASMO4', 'BIASMO5', 'BIASMO6', 'BIASMO7', 'BIASMO8', 'BIASMO9', 'BIASMO10'], inplace=True)

df_selected.head()

,GOFFRAC,CITY,STATECOD,OFFCOD,LOCCOD,BIASMO
0,B,Anchorage,AK,13A,38,11
1,I,Juneau,AK,13B,20,11
2,U,Juneau,AK,290,52,15
3,U,Anchorage,AK,290,4,82
4,B,Birmingham,AL,13C,20,11


In [ ]:
# Check for null values in the new columns
null_values_offcod = df_selected['OFFCOD'].isnull().sum()
null_values_loccod = df_selected['LOCCOD'].isnull().sum()
null_values_biasmo = df_selected['BIASMO'].isnull().sum()

# Print the results
print(f'Null values in OFFCOD column: {null_values_offcod}')
print(f'Null values in LOCCOD column: {null_values_loccod}')
print(f'Null values in BIASMO column: {null_values_biasmo}')

Null values in OFFCOD column: 0
Null values in LOCCOD column: 0
Null values in BIASMO column: 0


In [ ]:
# Dictionary mapping codes to values
offense_mapping = {
    '09A': 'Murder/non-neg manslaughter',
    '09B': 'Negligent manslaughter',
    '09C': 'Justifiable homicide',
    '100': 'Kidnapping/abduction',
    '11A': 'Rape',
    '11B': 'Sodomy',
    '11C': 'Sex asslt w/object',
    '11D': 'Fondling (Indecent Liberties/Child Molesting)',
    '120': 'Robbery',
    '13A': 'Agg asslt',
    '13B': 'Simple asslt',
    '13C': 'Intimidation',
    '200': 'Arson',
    '210': 'Extort/blackmail',
    '220': 'Burg/B & E',
    '23A': 'Pocket-picking',
    '23B': 'Purse-snatching',
    '23C': 'Shoplifting',
    '23D': 'Theft from building',
    '23E': 'Theft from coin-op machine',
    '23F': 'Theft from motor vehicle',
    '23G': 'Theft of mv parts',
    '23H': 'All other larceny',
    '240': 'Motor vehicle theft',
    '250': 'Counterfeit/forgery',
    '26A': 'False pretenses/swindle/conf game',
    '26B': 'Credit card/ATM fraud',
    '26C': 'Impersonation',
    '26D': 'Welfare fraud',
    '26E': 'Wire fraud',
    '26F': 'Identity Theft',
    '26G': 'Hacking/Computer Invasion',
    '270': 'Embezzlement',
    '280': 'Stolen property offenses',
    '290': 'Destruct/vandalism',
    '35A': 'Drug/Narc violations',
    '35B': 'Drug equip violations',
    '36A': 'Incest',
    '36B': 'Statutory rape',
    '370': 'Pornography/obscene material',
    '39A': 'Betting/wagering',
    '39B': 'Promote gambling',
    '39C': 'Gambling equip violations',
    '39D': 'Sports tampering',
    '40A': 'Prostitution',
    '40B': 'Assist prostitution',
    '40C': 'Purchasing prostitution',
    '510': 'Bribery',
    '520': 'Weapon law violations',
    '64A': 'Human Trafficking - Commercial sex acts',
    '64B': 'Human Trafficking - Involuntary servitude'
}

# Create the OFFCOD column using the mapping
df_selected['OFFCOD'] = df_selected['OFFCOD'].str.split(',').apply(lambda x: ', '.join([offense_mapping[code.strip()] for code in x]))
df_selected.head()

,GOFFRAC,CITY,STATECOD,OFFCOD,LOCCOD,BIASMO
0,B,Anchorage,AK,Agg asslt,38,11
1,I,Juneau,AK,Simple asslt,20,11
2,U,Juneau,AK,Destruct/vandalism,52,15
3,U,Anchorage,AK,Destruct/vandalism,4,82
4,B,Birmingham,AL,Intimidation,20,11


In [ ]:
# Check for NA values in the 'LOCCOD' column and count them
na_count = df_selected['LOCCOD'].isna().sum()

# Display the count of NA values
print(f"Number of NA values in the 'LOCCOD' column: {na_count}")

Number of NA values in the 'LOCCOD' column: 0


In [ ]:
# Remove rows with LOCCOD values '25' or '58'
df_selected = df_selected[~df_selected['LOCCOD'].isin(['25', '58'])]
# Initialize an empty dictionary to store counts
loc_cod_counts = {}

# Iterate through unique values in the 'LOCCOD' column
for value in df_selected['LOCCOD'].unique():
    # Count occurrences and store in the dictionary
    loc_cod_counts[value] = df_selected[df_selected['LOCCOD'] == value].shape[0]

# Display the counts
print("Unique values count in the 'LOCCOD' column:")
for value, count in loc_cod_counts.items():
    print(f"{value}: {count}")
# Mapping for LOCCOD
loccod_mapping = {
    '1': '0',
    '2': '0',
    '3': '0',
    '4': '0',
    '5': '0',
    '6': '1',
    '7': '0',
    '8': '0',
    '9': '0',
    '10': '0',
    '11': '0',
    '12': '0',
    '13': '0',
    '14': '1',
    '15': '0',
    '16': '0',
    '17': '0',
    '18': '0',
    '19': '0',
    '20': '1',
    '21': '0',
    '22': '0',
    '23': '0',
    '24': '0',
    '37': '0',
    '38': '0',
    '39': '0',
    '40': '0',
    '41': '1',
    '42': '0',
    '44': '1',
    '45': '0',
    '46': '1',
    '47': '0',
    '48': '1',
    '49': '1',
    '50': '0',
    '51': '0',
    '52': '0',
    '53': '0',
    '54': '0',
    '55': '0',
    '56': '1',
    '57': '0'
}

# Map the LOCCOD column using the mapping
df_selected['LOCCOD'] = df_selected['LOCCOD'].map(loccod_mapping)
# Check for NA values in the 'LOCCOD' column and count them
na_count = df_selected['LOCCOD'].isna().sum()

# Display the count of NA values
print(f"Number of NA values in the 'LOCCOD' column: {na_count}")
df_selected.head()

Unique values count in the 'LOCCOD' column:
38: 3
20: 2025
52: 223
4: 293
8: 93
13: 1217
5: 146
21: 199
12: 82
7: 108
11: 124
3: 103
22: 218
53: 344
15: 62
6: 28
18: 427
24: 77
44: 6
1: 110
23: 72
54: 14
2: 28
50: 125
14: 66
57: 24
9: 78
55: 23
10: 55
45: 3
39: 7
17: 13
51: 9
16: 13
46: 3
48: 12
42: 4
40: 2
47: 5
37: 5
41: 10
19: 13
49: 1
Number of NA values in the 'LOCCOD' column: 0


,GOFFRAC,CITY,STATECOD,OFFCOD,LOCCOD,BIASMO
0,B,Anchorage,AK,Agg asslt,0,11
1,I,Juneau,AK,Simple asslt,1,11
2,U,Juneau,AK,Destruct/vandalism,0,15
3,U,Anchorage,AK,Destruct/vandalism,0,82
4,B,Birmingham,AL,Intimidation,1,11


In [ ]:
# Convert the column from float to integer
df_selected['LOCCOD'] = df_selected['LOCCOD'].astype(int)
df_selected.head()

,GOFFRAC,CITY,STATECOD,OFFCOD,LOCCOD,BIASMO
0,B,Anchorage,AK,Agg asslt,0,11
1,I,Juneau,AK,Simple asslt,1,11
2,U,Juneau,AK,Destruct/vandalism,0,15
3,U,Anchorage,AK,Destruct/vandalism,0,82
4,B,Birmingham,AL,Intimidation,1,11


In [ ]:
# Mapping dictionary for BIASMO
biasmo_mapping = {
    '11': 'Anti-White',
    '12': 'Anti-Black or African American',
    '13': 'Anti-American Indian or Alaska Native',
    '14': 'Anti-Asian',
    '15': 'Anti-Multi-Racial',
    '16': 'Anti-Native Hawaiian or Other Pacific Islander',
    '21': 'Anti-Jewish',
    '22': 'Anti-Catholic',
    '23': 'Anti-Protestant',
    '24': 'Anti-Islamic',
    '25': 'Anti-Other Religion',
    '26': 'Anti-Multi-Religious',
    '27': 'Anti-Atheism/Agnosticism',
    '28': 'Anti-Mormon',
    '29': "Anti-Jehovah's Witness",
    '31': 'Anti-Arab',
    '32': 'Anti-Hispanic or Latino',
    '33': 'Anti-Not Hispanic or Latino',
    '41': 'Anti-Male Homosexual',
    '42': 'Anti-Female Homosexual',
    '43': 'Anti-Lesbian, Gay, Bisexual, or Transgender, Mixed Group (LGBT)',
    '44': 'Anti-Heterosexual',
    '45': 'Anti-Bisexual',
    '51': 'Anti-Physical Disability',
    '52': 'Anti-Mental Disability',
    '61': 'Anti-Male',
    '62': 'Anti-Female',
    '71': 'Anti-Transgender',
    '72': 'Anti-Gender Non-Conforming',
    '80': 'Undocumented Code',
    '81': 'Anti-Eastern Orthodox (Greek, Russian, etc)',
    '82': 'Anti-Other Christian',
    '83': 'Anti-Buddhist',
    '84': 'Anti-Hindu',
    '85': 'Anti-Sikh'
}

# Apply mapping to the 'BIASMO' column
df_selected['BIASMO'] = df_selected['BIASMO'].map(biasmo_mapping)

In [ ]:
# Assuming 'BIASMO' is the column of interest
unique_values = df_selected['BIASMO'].unique()

# Count and display unique values using a for loop
for value in unique_values:
    count = df_selected[df_selected['BIASMO'] == value].shape[0]
    print(f"{value}: {count}")

Anti-White: 657
Anti-Multi-Racial: 171
Anti-Other Christian: 21
Anti-Black or African American: 1860
Anti-Islamic: 251
Anti-Hispanic or Latino: 402
Anti-Mental Disability: 88
Anti-Male Homosexual: 605
Anti-Lesbian, Gay, Bisexual, or Transgender, Mixed Group (LGBT): 271
Anti-Heterosexual: 30
Anti-Not Hispanic or Latino: 247
Anti-Jewish: 733
Anti-American Indian or Alaska Native: 235
Anti-Physical Disability: 62
Anti-Other Religion: 80
Anti-Female Homosexual: 118
Anti-Transgender: 99
Anti-Asian: 105
Anti-Multi-Religious: 43
Anti-Catholic: 67
Anti-Bisexual: 22
Anti-Arab: 89
Anti-Native Hawaiian or Other Pacific Islander: 14
Anti-Sikh: 17
Anti-Protestant: 40
Anti-Gender Non-Conforming: 11
Anti-Female: 45
Anti-Hindu: 9
Anti-Jehovah's Witness: 6
Anti-Buddhist: 14
Anti-Mormon: 10
Anti-Atheism/Agnosticism: 6
Anti-Eastern Orthodox (Greek, Russian, etc): 20
Anti-Male: 25


In [ ]:
df_selected['Year'] = '2017'
# Keep only rows with STATECOD as 'CA'
df_selected_ca = df_selected[df_selected['STATECOD'] == 'CA']

In [ ]:
df_selected_ca.head()

,GOFFRAC,CITY,STATECOD,OFFCOD,LOCCOD,BIASMO,Year
306,U,Oakland,CA,Destruct/vandalism,0,Anti-Black or African American,2017
307,M,Oakland,CA,Agg asslt,0,Anti-Male Homosexual,2017
308,W,Alameda,CA,Simple asslt,0,Anti-Black or African American,2017
311,W,Alameda,CA,Destruct/vandalism,0,Anti-Jewish,2017
312,U,Alameda,CA,Destruct/vandalism,0,Anti-Black or African American,2017


In [ ]:
# Count rows before
rows_before = len(df_selected_ca)

# Drop rows with any NaN or blank values
df_selected_ca = df_selected_ca.dropna()

# Count rows after
rows_after = len(df_selected_ca)

print(f'Rows before: {rows_before}')
print(f'Rows after: {rows_after}')

Rows before: 1038
Rows after: 1038


In [ ]:
# Define the column renaming dictionary
column_rename_dict = {
    'OFFCOD': 'Type of Offence',
    'BIASMO': 'Bias Motivation',
    'GOFFRAC': 'Criminal Race',
    'LOCCOD': 'Type of Location',
    'STATECOD': 'State',
    'CITY': 'City',
    'Year': 'Year'
}

# Rename columns using the dictionary
df_selected_ca.rename(columns=column_rename_dict, inplace=True)
df_selected_ca.head()

,Criminal Race,City,State,Type of Offence,Type of Location,Bias Motivation,Year
306,U,Oakland,CA,Destruct/vandalism,0,Anti-Black or African American,2017
307,M,Oakland,CA,Agg asslt,0,Anti-Male Homosexual,2017
308,W,Alameda,CA,Simple asslt,0,Anti-Black or African American,2017
311,W,Alameda,CA,Destruct/vandalism,0,Anti-Jewish,2017
312,U,Alameda,CA,Destruct/vandalism,0,Anti-Black or African American,2017


In [ ]:
# Mapping for VictDescent
vict_descent_mapping = {
    'W': 'White',
    'B': 'Black or African American',
    'I': 'American Indian/Alaska Native',
    'A': 'Asian',
    'P': 'Native Hawaiian or other Pacific Islander',
    'M': 'Multi-Racial Group',
    'U': 'Unknown'
}

# Map values in VictDescent column
df_selected_ca['Criminal Race'] = df_selected_ca['Criminal Race'].map(vict_descent_mapping)

df_selected_ca['City'] = df_selected_ca['City'].apply(lambda x: x.title() if isinstance(x, str) else x)

In [ ]:
df_selected_ca['ID'] = ['HC' + str(i) for i in range(1, len(df_selected_ca) + 1)]

In [ ]:
# Save the DataFrame to a CSV file
df_selected_ca.to_csv('HateCrime.csv', index=False)

In [ ]:
# Download the CSV file
from google.colab import files
files.download('HateCrime.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>